In [1]:
import numpy as np
import timm
import torch
from selfViT import VisionTransformer

In [2]:
#helpers
def get_n_params(module):
        return sum(p.numel() for p in module.parameteres() if p.requires_grad)

In [3]:
def assert_tensors_equal(t1,t2):
    a1,a2=t1.detach().numpy(), t2.detach().numpy()
    
    np.testing.assert_allclose(a1,a2)

In [4]:
model_name="vit_base_patch16_384"
model_official=timm.create_model(model_name,pretrained=True);
model_official.eval()
print(type(model_official))

custom_config={
    "img_size":384,
    "in_chans":3,
    "patch_size":16,
    "embed_dim":768,
    "depth":12,
    "n_heads":12,
    "qkv_bias":True,
    "mlp_ratio":4.,
}
model_custom=VisionTransformer (**custom_config)
model_custom.eval()

<class 'timm.models.vision_transformer.VisionTransformer'>


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, 

In [5]:
for (n_o,p_o),(n_c,p_c) in zip(
                        model_official.named_parameters(),model_custom.named_parameters()
                        ):
    assert p_o.numel()==p_c.numel()
    print(f"{n_o} | {n_c}")
    p_c.data[:]=p_o.data
    assert_tensors_equal(p_c.data,p_o.data)

cls_token | cls_token
pos_embed | pos_embed
patch_embed.proj.weight | patch_embed.proj.weight
patch_embed.proj.bias | patch_embed.proj.bias
blocks.0.norm1.weight | blocks.0.norm1.weight
blocks.0.norm1.bias | blocks.0.norm1.bias
blocks.0.attn.qkv.weight | blocks.0.attn.qkv.weight
blocks.0.attn.qkv.bias | blocks.0.attn.qkv.bias
blocks.0.attn.proj.weight | blocks.0.attn.proj.weight
blocks.0.attn.proj.bias | blocks.0.attn.proj.bias
blocks.0.norm2.weight | blocks.0.norm2.weight
blocks.0.norm2.bias | blocks.0.norm2.bias
blocks.0.mlp.fc1.weight | blocks.0.mlp.fc1.weight
blocks.0.mlp.fc1.bias | blocks.0.mlp.fc1.bias
blocks.0.mlp.fc2.weight | blocks.0.mlp.fc2.weight
blocks.0.mlp.fc2.bias | blocks.0.mlp.fc2.bias
blocks.1.norm1.weight | blocks.1.norm1.weight
blocks.1.norm1.bias | blocks.1.norm1.bias
blocks.1.attn.qkv.weight | blocks.1.attn.qkv.weight
blocks.1.attn.qkv.bias | blocks.1.attn.qkv.bias
blocks.1.attn.proj.weight | blocks.1.attn.proj.weight
blocks.1.attn.proj.bias | blocks.1.attn.proj.b

In [8]:
inp=torch.rand(1,3,384,384)
type(inp)

res_c=model_custom(inp)
res_o=model_official(inp)

assert get_n_params(model_custom)==get_n_params(model_officai)
assert_tensors_equal(res_c,res_o)
# save custom model
torch.save(model_custom,"model.pth")

AttributeError: 'VisionTransformer' object has no attribute 'parameteres'